In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Kidney_Papillary_Cell_Carcinoma"
cohort = "GSE40911"

# Input paths
in_trait_dir = "../../input/GEO/Kidney_Papillary_Cell_Carcinoma"
in_cohort_dir = "../../input/GEO/Kidney_Papillary_Cell_Carcinoma/GSE40911"

# Output paths
out_data_file = "../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/GSE40911.csv"
out_gene_data_file = "../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/gene_data/GSE40911.csv"
out_clinical_data_file = "../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/clinical_data/GSE40911.csv"
json_path = "../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression analysis and in silico characterization of intronic long noncoding RNAs in renal cell carcinoma: emerging functional associations (RCC malignancy)"
!Series_summary	"Intronic and intergenic long noncoding RNAs (lncRNAs) are emerging gene expression regulators. The molecular pathogenesis of renal cell carcinoma (RCC) is still poorly understood, and in particular, limited studies are available for intronic lncRNAs expressed in RCC. Microarray experiments were performed with two different custom-designed arrays enriched with probes for lncRNAs mapping to intronic genomic regions. Samples from 18 primary clear cell RCC tumors and 11 nontumor adjacent matched tissues were analyzed with 4k-probes microarrays. Oligoarrays with 44k-probes were used to interrogate 17 RCC samples (14 clear cell, 2 papillary, 1 chromophobe subtypes) split into four pools. Meta-analyses were performed by taking the genomic coordinates of the RCC-expressed lncRNAs, a

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains expression data for long non-coding RNAs (lncRNAs)
# in renal cell carcinoma. This is gene expression data, not miRNA or methylation data.
import numpy as np
import pandas as pd
import os
import json

is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# For trait (renal cell carcinoma), we can use row 2 which has 'tissue: primary kidney tumor' and 'tissue: adjacent nontumor kidney tissue'
trait_row = 2

# For age, data is available in row 4
age_row = 4

# For gender, data is available in row 3
gender_row = 3

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert tissue type to binary (1 for tumor, 0 for non-tumor)"""
    if pd.isnull(value):
        return None
    value = value.lower()
    if 'tumor' in value and 'non' not in value and 'adjacent' not in value:
        return 1  # Tumor
    elif ('nontumor' in value) or ('adjacent' in value):
        return 0  # Non-tumor
    return None

def convert_age(value):
    """Convert age to continuous numeric value"""
    if pd.isnull(value):
        return None
    try:
        # Extract the number after "age at surgery (yrs): "
        age_str = value.split(':')[1].strip()
        return float(age_str)
    except (ValueError, IndexError):
        return None

def convert_gender(value):
    """Convert gender to binary (0 for female, 1 for male)"""
    if pd.isnull(value):
        return None
    value = value.lower()
    if 'female' in value:
        return 0
    elif 'male' in value:
        return 1
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Save metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# If trait data is available, extract clinical features
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary provided in the previous step
    # The dictionary keys are row indices and values are lists of unique values
    sample_chars_dict = {
        0: ['patient identifier: 3', 'patient identifier: 5', 'patient identifier: 8', 'patient identifier: 9', 'patient identifier: 10', 'patient identifier: 11', 'patient identifier: 24', 'patient identifier: 26', 'patient identifier: 28', 'patient identifier: 30', 'patient identifier: 31'],
        1: ['disease: clear cell renal cell carcinoma (RCC)'],
        2: ['tissue: adjacent nontumor kidney tissue', 'tissue: primary kidney tumor'],
        3: ['gender: female', 'gender: male'],
        4: ['age at surgery (yrs): 78', 'age at surgery (yrs): 53', 'age at surgery (yrs): 71', 'age at surgery (yrs): 39', 'age at surgery (yrs): 34', 'age at surgery (yrs): 51', 'age at surgery (yrs): 75', 'age at surgery (yrs): 40', 'age at surgery (yrs): 50'],
        5: ['patient status: cancer-specific death', 'patient status: dead from other causes', 'patient status: alive without cancer', 'patient status: alive with cancer', 'fuhrman grade: IV', 'fuhrman grade: III', 'fuhrman grade: II'],
        6: [np.nan, 'tumor size (cm): 6', 'tumor size (cm): 8', 'tumor size (cm): 5', 'tumor size (cm): 6.5', 'tumor size (cm): 7', 'tumor size (cm): 15', 'tumor size (cm): 8.5'],
        7: [np.nan, 'necrosis: yes', 'necrosis: no'],
        8: [np.nan, 'capsule infiltration: yes', 'capsule infiltration: no'],
        9: [np.nan, 'tnm classification (t): 3c', 'tnm classification (t): 2', 'tnm classification (t): 3a', 'tnm classification (t): 1b', 'tnm classification (t): 3b', 'tnm classification (t): 1'],
        10: [np.nan, 'tnm classification (n): no data available', 'tnm classification (n): 1', 'tnm classification (n): 0', 'tnm classification (n): 2'],
        11: [np.nan, 'tnm classification (m): 1', 'tnm classification (m): no data available'],
        12: [np.nan, 'organ metastasis at surgery: endocava, bones', 'organ metastasis at surgery: liver', 'organ metastasis at surgery: no data available', 'organ metastasis at surgery: lung, adjacent tissues'],
        13: [np.nan, 'organ metastasis after surgery: no data available', 'organ metastasis after surgery: liver, spleen', 'organ metastasis after surgery: brain, lung, bones'],
        14: [np.nan, 'patient status: cancer-specific death', 'patient status: dead from other causes', 'patient status: alive without cancer', 'patient status: alive with cancer']
    }
    
    # Convert the dictionary to a DataFrame
    clinical_data = pd.DataFrame.from_dict(sample_chars_dict, orient='index')
    
    # Extract clinical features
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    print("Clinical features preview:")
    print(preview_df(clinical_features))
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical features to a CSV file
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to: {out_clinical_data_file}")


Clinical features preview:
{0: [0.0, 78.0, 0.0], 1: [1.0, 53.0, 1.0], 2: [nan, 71.0, nan], 3: [nan, 39.0, nan], 4: [nan, 34.0, nan], 5: [nan, 51.0, nan], 6: [nan, 75.0, nan], 7: [nan, 40.0, nan], 8: [nan, 50.0, nan], 9: [nan, nan, nan], 10: [nan, nan, nan]}
Clinical features saved to: ../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/clinical_data/GSE40911.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Add diagnostic code to check file content and structure
print("Examining matrix file structure...")
with gzip.open(matrix_file, 'rt') as file:
    table_marker_found = False
    lines_read = 0
    for i, line in enumerate(file):
        lines_read += 1
        if '!series_matrix_table_begin' in line:
            table_marker_found = True
            print(f"Found table marker at line {i}")
            # Read a few lines after the marker to check data structure
            next_lines = [next(file, "").strip() for _ in range(5)]
            print("First few lines after marker:")
            for next_line in next_lines:
                print(next_line)
            break
        if i < 10:  # Print first few lines to see file structure
            print(f"Line {i}: {line.strip()}")
        if i > 100:  # Don't read the entire file
            break
    
    if not table_marker_found:
        print("Table marker '!series_matrix_table_begin' not found in first 100 lines")
    print(f"Total lines examined: {lines_read}")

# 2. Try extracting gene expression data from the matrix file again with better diagnostics
try:
    print("\nAttempting to extract gene data from matrix file...")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {str(e)}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")

# If data extraction failed, try an alternative approach using pandas directly
if not is_gene_available:
    print("\nTrying alternative approach to read gene expression data...")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            # Skip lines until we find the marker
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Try to read the data directly with pandas
            gene_data = pd.read_csv(file, sep='\t', index_col=0)
            
            if not gene_data.empty:
                print(f"Successfully extracted gene data with alternative method: {gene_data.shape}")
                print("First 20 gene IDs:")
                print(gene_data.index[:20])
                is_gene_available = True
            else:
                print("Alternative extraction method also produced empty data")
    except Exception as e:
        print(f"Alternative extraction failed: {str(e)}")


Examining matrix file structure...
Line 0: !Series_title	"Expression analysis and in silico characterization of intronic long noncoding RNAs in renal cell carcinoma: emerging functional associations (RCC malignancy)"
Line 1: !Series_geo_accession	"GSE40911"
Line 2: !Series_status	"Public on Jan 21 2014"
Line 3: !Series_submission_date	"Sep 14 2012"
Line 4: !Series_last_update_date	"Jan 21 2014"
Line 5: !Series_pubmed_id	"24238219"
Line 6: !Series_summary	"Intronic and intergenic long noncoding RNAs (lncRNAs) are emerging gene expression regulators. The molecular pathogenesis of renal cell carcinoma (RCC) is still poorly understood, and in particular, limited studies are available for intronic lncRNAs expressed in RCC. Microarray experiments were performed with two different custom-designed arrays enriched with probes for lncRNAs mapping to intronic genomic regions. Samples from 18 primary clear cell RCC tumors and 11 nontumor adjacent matched tissues were analyzed with 4k-probes microa

### Step 4: Gene Identifier Review

In [5]:
# Gene Identifier Review
# Examining the gene IDs in the extracted data

# Based on the data shown, these identifiers appear to be probe IDs or numeric identifiers
# rather than standard human gene symbols.
# Human gene symbols would typically be alphanumeric (like BRCA1, TP53, etc.)
# The identifiers shown are simple numbers like '3', '4', '6', '7', etc.
# These would need to be mapped to actual gene symbols for meaningful analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 139072 rows

Gene annotation preview (first few rows):
{'ID': ['910', '4260', '1981', '2381', '4288'], 'GB_ACC': ['BE833259', 'BE702227', 'BF364095', 'BE081005', 'AW880607'], 'SPOT_TYPE': ['Exonic', 'Exonic', 'Exonic', 'Exonic', 'Exonic'], 'GENE_ID': [85439.0, 2776.0, 84131.0, 2776.0, 54768.0], 'GENE_SYMBOL': ['STON2', 'GNAQ', 'CEP78', 'GNAQ', 'HYDIN'], 'GENE_ANNOTATION': ['stonin 2', 'Guanine nucleotide binding protein (G protein), q polypeptide', 'centrosomal protein 78kDa', 'Guanine nucleotide binding protein (G protein), q polypeptide', 'hydrocephalus inducing homolog 2 (mouse); hydrocephalus inducing homolog (mouse)'], 'CPC_CODING_POTENTIAL': ['noncoding', 'noncoding', 'noncoding', 'noncoding', '-'], 'SEQUENCE': ['CTGATCCGCTTAAGCTTAGTATGTTTGAGTGTGTAATTTTAGTTTCTTTTCTGGTTGTATTTGTGGTAGTCAGATGTGTTGGATTGATTCCAACTGGACAGAGTAAGGAATTCCAGCATCCTCTTCCTGCTTGCTCGTGTTACCCCACAGATCAAACCCTCAATTCTAGTTGGGGATGCTGTCTAGCCCCACACCATGACTGAAGCCTTAAGCACTGTTGCG

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Examine which annotation column contains the probe IDs that match the gene expression data
# Based on previous analysis, the gene expression data has numeric IDs like '3', '4', '6'
# Looking at the annotation data, the 'ID' column seems to contain these identifiers

# Check the first few IDs from both datasets to verify the match
print("\nComparing gene IDs between datasets:")
print("First few IDs in gene expression data:", gene_data.index[:5].tolist())
print("First few IDs in gene annotation data:", gene_annotation['ID'][:5].tolist())

# 2. Extract the gene mapping dataframe
# We need to map probe IDs to gene symbols, where:
# - ID column in gene_annotation matches the gene_data index
# - GENE_SYMBOL column contains the gene symbols we want to map to
prob_col = 'ID'
gene_col = 'GENE_SYMBOL'

# Create the mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"\nCreated gene mapping with {len(gene_mapping)} rows")
print("Gene mapping preview:")
print(preview_df(gene_mapping))

# 3. Apply the gene mapping to convert probe-level data to gene-level data
print("\nConverting probe-level measurements to gene expression data...")

# Apply the mapping with the library function
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Converted gene expression data has {len(gene_data.index)} genes")
print("Gene data preview:")
print(preview_df(gene_data))

# Save the gene expression data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to: {out_gene_data_file}")



Comparing gene IDs between datasets:
First few IDs in gene expression data: ['3', '4', '6', '7', '9']
First few IDs in gene annotation data: ['910', '4260', '1981', '2381', '4288']



Created gene mapping with 3169 rows
Gene mapping preview:
{'ID': ['910', '4260', '1981', '2381', '4288'], 'Gene': ['STON2', 'GNAQ', 'CEP78', 'GNAQ', 'HYDIN']}

Converting probe-level measurements to gene expression data...
Converted gene expression data has 2502 genes
Gene data preview:
{'GSM1004655': [11.611, 10.926954546000001, 5.866136364, 2.304568182, 3.128022727], 'GSM1004656': [12.39861364, 15.123068182, 6.7795, 2.531636364, 4.258579545], 'GSM1004657': [8.044136364, 9.524568182, 7.152886364, 4.169409091, 5.121579545], 'GSM1004658': [8.556590909, 9.290693181, 7.174977273, 3.884727273, 5.776840909], 'GSM1004659': [7.704806818, 16.374318182, 5.923522727, 0.992409091, 5.195102273], 'GSM1004660': [6.522909091, 9.687295455000001, 7.814090909, 2.001590909, 6.675045455], 'GSM1004661': [9.999795455, 10.968193182, 5.647227273, 2.02575, 5.629681818], 'GSM1004662': [9.7725, 12.251704546, 5.936204545, 4.709954545, 5.837704545], 'GSM1004663': [7.132090909, 9.254681818, 5.132613636, 2.18736363

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("\nNormalizing gene symbols...")
try:
    # Load gene data that was saved in step 6
    gene_data = pd.read_csv(out_gene_data_file, index_col=0)
    print(f"Loaded gene data with {gene_data.shape[0]} genes and {gene_data.shape[1]} samples")
    
    # Normalize gene symbols using NCBI Gene database
    gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"After normalization: {gene_data.shape[0]} genes")
    
    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    
    is_gene_available = True
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    is_gene_available = False

# 2. Link clinical and genetic data
print("\nLinking clinical and genetic data...")
try:
    # Load clinical data from step 2
    clinical_data = pd.read_csv(out_clinical_data_file)
    print(f"Loaded clinical data with shape: {clinical_data.shape}")
    
    # Print clinical data columns and preview for debugging
    print("Clinical data columns:", clinical_data.columns.tolist())
    print("Clinical data preview:")
    print(clinical_data.head())
    
    # In clinical data from geo_select_clinical_features, the trait is the first column (index 0)
    # and rows are in columns format (with genes as rows in gene_data)
    # We need to transpose it for linking with gene data
    clinical_transpose = clinical_data.T
    
    # Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_transpose, gene_data)
    print(f"Created linked data with {linked_data.shape[0]} samples and {linked_data.shape[1]} features")
    is_trait_available = True
except Exception as e:
    print(f"Error linking clinical and genetic data: {e}")
    is_trait_available = False
    linked_data = pd.DataFrame()

# 3. Handle missing values in the linked data
if is_trait_available and not linked_data.empty:
    print("\nHandling missing values...")
    try:
        # Rename the first column to the trait name for consistency
        if linked_data.columns[0] != trait:
            linked_data = linked_data.rename(columns={linked_data.columns[0]: trait})
        
        linked_data = handle_missing_values(linked_data, trait)
        print(f"After handling missing values: {linked_data.shape[0]} samples and {linked_data.shape[1]} features")
    except Exception as e:
        print(f"Error handling missing values: {e}")
        
    # 4. Determine whether the trait and demographic features are biased
    print("\nEvaluating feature bias...")
    try:
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        print(f"Trait bias determination: {is_biased}")
        print(f"Final linked data shape: {linked_data.shape[0]} samples and {linked_data.shape[1]} features")
    except Exception as e:
        print(f"Error evaluating feature bias: {e}")
        is_biased = True
else:
    print("\nSkipping missing value handling and bias evaluation as linked data is not available")
    is_biased = True

# 5. Validate and save cohort information
print("\nPerforming final validation...")
note = ""
if not is_trait_available:
    note = "Dataset does not contain required trait information"
elif is_biased:
    note = "Dataset has severe bias in the trait distribution"

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. Save the linked data if usable
print(f"\nDataset usability for {trait} association studies: {is_usable}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Final linked data saved to {out_data_file}")
else:
    if note:
        print(f"Reason: {note}")
    else:
        print("Dataset does not meet quality criteria for the specified trait")


Normalizing gene symbols...
Loaded gene data with 2502 genes and 44 samples
After normalization: 2488 genes


Normalized gene data saved to ../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/gene_data/GSE40911.csv

Linking clinical and genetic data...
Loaded clinical data with shape: (3, 12)
Clinical data columns: ['Unnamed: 0', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
Clinical data preview:
                        Unnamed: 0     0     1     2     3     4     5     6  \
0  Kidney_Papillary_Cell_Carcinoma   0.0   1.0   NaN   NaN   NaN   NaN   NaN   
1                              Age  78.0  53.0  71.0  39.0  34.0  51.0  75.0   
2                           Gender   0.0   1.0   NaN   NaN   NaN   NaN   NaN   

      7     8   9  10  
0   NaN   NaN NaN NaN  
1  40.0  50.0 NaN NaN  
2   NaN   NaN NaN NaN  
Created linked data with 47 samples and 2500 features

Handling missing values...
After handling missing values: 3 samples and 3 features

Evaluating feature bias...
Error evaluating feature bias: unsupported operand type(s) for -: 'str' and 'str'

Performing final validation..

/media/techt/DATA/GenoAgent/tools/preprocess.py:455: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[gene_cols] = df[gene_cols].fillna(df[gene_cols].mean())
